In [1]:
import pandas as pd
import numpy as np
import re 
from fuzzywuzzy import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
#import cleanco
import functools
import operator
from collections import OrderedDict
import re
import unicodedata

## Recreate Cleanco

In [2]:
#Term data
terms_by_type = {
   'Corporation': ['company', 'incorporated', 'corporation', 'corp.', 'corp', 'inc',
      '& co.', '& co', 'inc.', 's.p.a.', 'n.v.', 'a.g.', 'ag', 'nuf', 's.a.', 's.f.',
      'oao', 'co.', 'co'
   ],
   'General Partnership': ['soc.col.', 'stg', 'd.n.o.', 'ltda.', 'v.o.s.', 'a spol.',
      u've\xc5\x99. obch. spol.', 'kgaa', 'o.e.', 's.f.', 's.n.c.', 's.a.p.a.', 'j.t.d.',
      'v.o.f.', 'sp.j.', 'og', 'sd', ' i/s', 'ay', 'snc', 'oe', 'bt.', 's.s.', 'mb',
      'ans', 'da', 'o.d.', 'hb', 'pt'
   ],
   'Joint Stock / Unlimited': ['unltd', 'ultd', 'sal', 'unlimited', 'saog', 'saoc', 'aj',
      'yoaj', 'oaj', 'akc. spol.', 'a.s.'
   ],
   'Joint Venture': ['esv', 'gie', 'kv.', 'qk'],
   'Limited': ['pty. ltd.', 'pty ltd', 'ltd', 'l.t.d.', 'bvba', 'd.o.o.', 'ltda', 'gmbh',
      'g.m.b.h', 'kft.', 'kht.', 'zrt.', 'ehf.', 's.a.r.l.', 'd.o.o.e.l.', 's. de r.l.',
      'b.v.', 'tapui',
      'sp. z.o.o.', 'sp. z o.o.', 'spółka z o.o.',
      's.r.l.', 's.l.', 's.l.n.e.', 'ood', 'oy', 'rt.',
      'teo', 'uab', 'scs', 'sprl', 'limited', 'bhd.', 'sdn. bhd.', 'sdn bhd', 'as',
      'lda.', 'tov', 'pp'
   ],
   'Limited Liability Company': ['pllc', 'llc', 'l.l.c.', 'plc.', 'plc', 'hf.', 'oyj',
      'a.e.', 'nyrt.', 'p.l.c.', 'sh.a.', 's.a.', 's.r.l.', 'srl.', 'srl', 'aat', '3at', 'd.d.',
      's.r.o.', 'spol. s r.o.', 's.m.b.a.', 'smba', 'sarl', 'nv', 'sa', 'aps',
      'a/s', 'p/s', 'sae', 'sasu', 'eurl', 'ae', 'cpt', 'as', 'ab', 'asa', 'ooo', 'dat',
      'vat', 'zat', 'mchj', 'a.d.'
   ],
   'Limited Liability Limited Partnership': ['lllp', 'l.l.l.p.'],
   'Limited Liability Partnership': ['llp', 'l.l.p.', 'sp.p.', 's.c.a.', 's.c.s.'],
   'Limited Partnership': ['gmbh & co. kg', 'lp', 'l.p.', 's.c.s.',
      's.c.p.a', 'comm.v', 'k.d.', 'k.d.a.', 's. en c.', 'e.e.', 's.a.s.', 's. en c.',
      'c.v.', 's.k.a.', 'sp.k.', 's.cra.', 'ky', 'scs', 'kg', 'kd', 'k/s', 'ee', 'secs',
      'kda', 'ks', 'kb','kt'
   ],
   'Mutual Fund': ['sicav'],
   'No Liability': ['nl'],
   'Non-Profit': ['vzw', 'ses.', 'gte.'],
   'Private Company': ['private', 'pte', 'xk'],
   'Professional Corporation': ['p.c.', 'vof', 'snc'],
   'Professional Limited Liability Company': ['pllc', 'p.l.l.c.'],
   'Sole Proprietorship': ['e.u.', 's.p.', 't:mi', 'tmi', 'e.v.', 'e.c.', 'et', 'obrt',
      'fie', 'ij', 'fop', 'xt'
   ]
}

terms_by_country = {
   'Albania': ['sh.a.', 'sh.p.k.'],
   'Argentina': ['s.a.', 's.r.l.', 's.c.p.a', 'scpa', 's.c.e i.', 's.e.', 's.g.r',
      'soc.col.'
   ],
   'Australia': ['nl', 'pty. ltd.', 'pty ltd'],
   'Austria': ['e.u.', 'stg', 'gesbr', 'a.g.', 'ag', 'og', 'kg'],
   'Belarus': ['aat', '3at'],
   'Belgium': ['esv', 'vzw', 'vof', 'snc', 'comm.v', 'scs', 'bvba', 'sprl', 'cvba',
      'cvoa', 'sca', 'sep', 'gie'
   ],
   'Bosnia / Herzegovina': ['d.d.', 'a.d.', 'd.n.o.', 'd.o.o.', 'k.v.', 's.p.'],
   'Brazil': ['ltda', 's.a.', 'pllc', 'ad', 'adsitz', 'ead', 'et', 'kd', 'kda', 'sd'],
   'Bulgaria': ['ad', 'adsitz', 'ead', 'et', 'kd', 'kda', 'sd'],
   'Cambodia': ['gp', 'sm pte ltd.', 'pte ltd.', 'plc ltd.', 'peec', 'sp'],
   'Canada': ['gp', 'lp', 'sp'],
   'Chile': ['eirl', 's.a.', 'sgr', 's.g.r.', 'ltda', 's.p.a.', 'sa', 's. en c.',
      'ltda.'
   ],
   'Columbia': ['s.a.', 'e.u.', 's.a.s.', 'suc. de descendants', 'sca'],
   'Croatia': ['d.d.', 'd.o.o.', 'obrt'],
   'Czech Republic': ['a.s.', 'akc. spol.', 's.r.o.', 'spol. s r.o.', 'v.o.s.', u've\xc5\x99. obch. spol.', 'a spol.', 'k.s.', 'kom. spol.', 'kom. spol.'],
   'Denmark': ['i/s', 'a/s', 'k/s', 'p/s', 'amba', 'a.m.b.a.', 'fmba', 'f.m.b.a.', 'smba',
      's.m.b.a.', 'g/s'
   ],
   'Dominican Republic': ['c. por a.', 'cxa', 's.a.', 's.a.s.', 'srl.', 'srl', 'eirl.', 'sa',
      'sas'
   ],
   'Ecuador': ['s.a.', 'c.a.', 'sa', 'ep'],
   'Egypt': ['sae'],
   'Estonia': ['fie'],
   'Finland': ['t:mi', 'tmi', 'as oy', 'as.oy', 'ay', 'ky', 'oy', 'oyj', 'ok'],
   'France': ['sicav', 'sarl', 'sogepa', 'ei', 'eurl', 'sasu', 'fcp', 'gie', 'sep', 'snc',
      'scs', 'sca', 'scop', 'sem', 'sas'
   ],
   'Germany': ['gmbh & co. kg', 'e.g.', 'e.v.', 'gbr', 'ohg', 'partg',
      'kgaa', 'gmbh', 'g.m.b.h.', 'ag', 'mbh & co. kg'
   ],
   'Greece': ['a.e.', 'ae', 'e.e.', 'ee', 'epe', 'e.p.e.', 'mepe', 'm.e.p.e.', 'o.e.',
      'oe', 'ovee', 'o.v.e.e.'
   ],
   'Guatemala': ['s.a.', 'sa'],
   'Haiti': ['sa'],
   'Hong Kong': ['ltd', 'unltd', 'ultd', 'limited'],
   'Hungary': ['e.v.', 'e.c.', 'bt.', 'kft.', 'kht.', 'kkt.', 'k.v.', 'zrt.', 'nyrt',
      'ev', 'ec', 'rt.'
   ],
   'Iceland': ['ehf.', 'hf.', 'ohf.', 's.f.', 'ses.'],
   'India': ['pvt. ltd.', 'ltd.', 'psu', 'pse'],
   'Indonesia': ['ud', 'fa', 'pt'],
   'Ireland': ['cpt', 'teo'],
   'Israel': ['b.m.', 'bm', 'ltd', 'limited'],
   'Italy': ['s.n.c.', 's.a.s.', 's.p.a.', 's.a.p.a.', 's.r.l.', 's.c.r.l.', 's.s.'],
   'Latvia': ['as', 'sia', 'ik', 'ps', 'ks'],
   'Lebanon': ['sal'],
   'Lithuania': ['uab', 'ab', 'ij', 'mb'],
   'Luxemborg': ['s.a.', 's.a.r.l.', 'secs'],
   'Macedonia': ['d.o.o.', 'd.o.o.e.l', 'k.d.a.', 'j.t.d.', 'a.d.', 'k.d.'],
   'Malaysia': ['bhd.', 'sdn. bhd.'],
   'Mexico': ['s.a.', 's. de. r.l.', 's. en c.', 's.a.b.', 's.a.p.i.'],
   'Mongolia': ['xk', 'xxk'],
   'Netherlands': ['v.o.f.', 'c.v.', 'b.v.', 'n.v.'],
   'New Zealand': ['tapui', 'ltd', 'limited'],
   'Nigeria': ['gte.', 'plc', 'ltd.', 'ultd.'],
   'Norway': ['asa', 'as', 'ans', 'ba', 'bl', 'da', 'etat', 'fkf', 'hf', 'iks', 'kf',
      'ks', 'nuf', 'rhf', 'sf'
   ],
   'Oman': ['saog', 'saoc'],
   'Pakistan': ['ltd.', 'pvt. ltd.', 'ltd', 'limited'],
   'Peru': ['sa', 's.a.', 's.a.a.'],
   'Philippines': ['coop.', 'corp.', 'corp', 'ent.', 'inc.', 'inc', 'llc', 'l.l.c.',
      'ltd.'
   ],
   'Poland': ['p.p.', 's.k.a.', 'sp.j.', 'sp.k.', 'sp.p.', 'sp. z.o.o.', 's.c.', 's.a.'],
   'Portugal': ['lda.', 'crl', 's.a.', 's.f.', 'sgps'],
   'Romania': ['s.c.a.', 's.c.s.', 's.n.c.', 's.r.l.', 'o.n.g.', 's.a.'],
   'Russia': ['ooo', 'oao', 'zao', '3ao', 'пао', 'оао', 'ооо'],
   'Serbia': ['d.o.o.', 'a.d.', 'k.d.', 'o.d.'],
   'Singapore': ['bhd', 'pte ltd', 'sdn bhd', 'llp', 'l.l.p.', 'ltd.', 'pte', 'pte. ltd.'],
   'Slovenia': ['d.d.', 'd.o.o.', 'd.n.o.', 'k.d.', 's.p.'],
   'Slovakia': ['a.s.', 'akc. spol.', 's.r.o.', 'spol. s r.o.', 'k.s.', 'kom. spol.', 'v.o.s.', 'a spol.'],
   'Spain': ['s.a.', 's.a.d.', 's.l.', 's.l.l.', 's.l.n.e.', 's.c.', 's.cra', 's.coop',
      'sal', 'sccl'
   ],
   'Sweden': ['ab', 'hb', 'kb'],
   'Switzerland': ['ab', 'sa', 'gmbh', 'g.m.b.h.', 'sarl', 'sagl'],
   'Turkey': ['koop.'],
   'Ukraine': ['dat', 'fop', 'kt', 'pt', 'tdv', 'tov', 'pp', 'vat', 'zat', 'at'],
   'United Kingdom': ['plc.', 'plc', 'cic', 'cio', 'l.l.p.', 'llp', 'l.p.', 'lp', 'ltd.',
      'ltd', 'limited'
   ],
   'United States of America': ['llc', 'inc.', 'corporation', 'incorporated', 'company',
      'limited', 'corp.', 'inc.', 'inc', 'llp', 'l.l.p.', 'pllc', 'and company',
      '& company', 'inc', 'inc.', 'corp.', 'corp', 'ltd.', 'ltd', '& co.', '& co', 'co.',
      'co', 'lp'
   ],
   'Uzbekistan': ['mchj', 'qmj', 'aj', 'oaj', 'yoaj', 'xk', 'xt', 'ok', 'uk', 'qk']
}

In [3]:
# from https://stackoverflow.com/a/51230541
NON_NFKD_MAP = {
    u"\u0181": u"B",
    u"\u1d81": u"d",
    u"\u1d85": u"l",
    u"\u1d89": u"r",
    u"\u028b": u"v",
    u"\u1d8d": u"x",
    u"\u1d83": u"g",
    u"\u0191": u"F",
    u"\u0199": u"k",
    u"\u019d": u"N",
    u"\u0220": u"N",
    u"\u01a5": u"p",
    u"\u0224": u"Z",
    u"\u0126": u"H",
    u"\u01ad": u"t",
    u"\u01b5": u"Z",
    u"\u0234": u"l",
    u"\u023c": u"c",
    u"\u0240": u"z",
    u"\u0142": u"l",
    u"\u0244": u"U",
    u"\u2c60": u"L",
    u"\u0248": u"J",
    u"\ua74a": u"O",
    u"\u024c": u"R",
    u"\ua752": u"P",
    u"\ua756": u"Q",
    u"\ua75a": u"R",
    u"\ua75e": u"V",
    u"\u0260": u"g",
    u"\u01e5": u"g",
    u"\u2c64": u"R",
    u"\u0166": u"T",
    u"\u0268": u"i",
    u"\u2c66": u"t",
    u"\u026c": u"l",
    u"\u1d6e": u"f",
    u"\u1d87": u"n",
    u"\u1d72": u"r",
    u"\u2c74": u"v",
    u"\u1d76": u"z",
    u"\u2c78": u"e",
    u"\u027c": u"r",
    u"\u1eff": u"y",
    u"\ua741": u"k",
    u"\u0182": u"B",
    u"\u1d86": u"m",
    u"\u0288": u"t",
    u"\u018a": u"D",
    u"\u1d8e": u"z",
    u"\u0111": u"d",
    u"\u0290": u"z",
    u"\u0192": u"f",
    u"\u1d96": u"i",
    u"\u019a": u"l",
    u"\u019e": u"n",
    u"\u1d88": u"p",
    u"\u02a0": u"q",
    u"\u01ae": u"T",
    u"\u01b2": u"V",
    u"\u01b6": u"z",
    u"\u023b": u"C",
    u"\u023f": u"s",
    u"\u0141": u"L",
    u"\u0243": u"B",
    u"\ua745": u"k",
    u"\u0247": u"e",
    u"\ua749": u"l",
    u"\u024b": u"q",
    u"\ua74d": u"o",
    u"\u024f": u"y",
    u"\ua751": u"p",
    u"\u0253": u"b",
    u"\ua755": u"p",
    u"\u0257": u"d",
    u"\ua759": u"q",
    u"\xd8": u"O",
    u"\u2c63": u"P",
    u"\u2c67": u"H",
    u"\u026b": u"l",
    u"\u1d6d": u"d",
    u"\u1d71": u"p",
    u"\u0273": u"n",
    u"\u1d75": u"t",
    u"\u1d91": u"d",
    u"\xf8": u"o",
    u"\u2c7e": u"S",
    u"\u1d7d": u"p",
    u"\u2c7f": u"Z",
    u"\u0183": u"b",
    u"\u0187": u"C",
    u"\u1d80": u"b",
    u"\u0289": u"u",
    u"\u018b": u"D",
    u"\u1d8f": u"a",
    u"\u0291": u"z",
    u"\u0110": u"D",
    u"\u0193": u"G",
    u"\u1d82": u"f",
    u"\u0197": u"I",
    u"\u029d": u"j",
    u"\u019f": u"O",
    u"\u2c6c": u"z",
    u"\u01ab": u"t",
    u"\u01b3": u"Y",
    u"\u0236": u"t",
    u"\u023a": u"A",
    u"\u023e": u"T",
    u"\ua740": u"K",
    u"\u1d8a": u"s",
    u"\ua744": u"K",
    u"\u0246": u"E",
    u"\ua748": u"L",
    u"\ua74c": u"O",
    u"\u024e": u"Y",
    u"\ua750": u"P",
    u"\ua754": u"P",
    u"\u0256": u"d",
    u"\ua758": u"Q",
    u"\u2c62": u"L",
    u"\u0266": u"h",
    u"\u2c73": u"w",
    u"\u2c6a": u"k",
    u"\u1d6c": u"b",
    u"\u2c6e": u"M",
    u"\u1d70": u"n",
    u"\u0272": u"n",
    u"\u1d92": u"e",
    u"\u1d74": u"s",
    u"\u2c7a": u"o",
    u"\u2c6b": u"Z",
    u"\u027e": u"r",
    u"\u0180": u"b",
    u"\u0282": u"s",
    u"\u1d84": u"k",
    u"\u0188": u"c",
    u"\u018c": u"d",
    u"\ua742": u"K",
    u"\u1d99": u"u",
    u"\u0198": u"K",
    u"\u1d8c": u"v",
    u"\u0221": u"d",
    u"\u2c71": u"v",
    u"\u0225": u"z",
    u"\u01a4": u"P",
    u"\u0127": u"h",
    u"\u01ac": u"T",
    u"\u0235": u"n",
    u"\u01b4": u"y",
    u"\u2c72": u"W",
    u"\u023d": u"L",
    u"\ua743": u"k",
    u"\u0249": u"j",
    u"\ua74b": u"o",
    u"\u024d": u"r",
    u"\ua753": u"p",
    u"\u0255": u"c",
    u"\ua757": u"q",
    u"\u2c68": u"h",
    u"\ua75b": u"r",
    u"\ua75f": u"v",
    u"\u2c61": u"l",
    u"\u2c65": u"a",
    u"\u01e4": u"G",
    u"\u0167": u"t",
    u"\u2c69": u"K",
    u"\u026d": u"l",
    u"\u1d6f": u"m",
    u"\u0271": u"m",
    u"\u1d73": u"r",
    u"\u027d": u"r",
    u"\u1efe": u"Y",
}

In [4]:
def typesources():
    types = []
    for business_type in terms_by_type:
        for item in terms_by_type[business_type]:
            types.append((business_type, item))

    return sorted(types, key=lambda part: len(part[1]), reverse=True)


def countrysources():
    countries = []
    for country in terms_by_country:
        for item in terms_by_country[country]:
            countries.append((country, item))

    return sorted(countries, key=lambda part: len(part[1]), reverse=True)


def matches(name, sources):
    name = strip_tail(name)
    parts = name.split()
    nparts = [normalized(p) for p in parts]
    matches = []
    for classifier, term in sources:
        nterm = normalized(term)
        try:
            idx = nparts.index(nterm)
        except ValueError:
            pass
        else:
            matches.append(classifier)

    return matches

In [5]:
tail_removal_rexp = re.compile(r"[^\.\w\(\)]+$", flags=re.UNICODE)


def get_unique_terms():
    "retrieve all unique terms from termdata definitions"
    ts = functools.reduce(operator.iconcat, terms_by_type.values(), [])
    cs = functools.reduce(operator.iconcat, terms_by_country.values(), [])
    return set(ts + cs)


def remove_accents(t):
    """based on https://stackoverflow.com/a/51230541"""
    nfkd_form = unicodedata.normalize('NFKD', t.casefold())
    return ''.join(
        NON_NFKD_MAP[c]
            if c in NON_NFKD_MAP
        else c
            for part in nfkd_form for c in part
            if unicodedata.category(part) != 'Mn'
        )


def strip_punct(t):
    return t.replace(".", "").replace(",", "").replace("-", "")


def normalize_terms(terms):
    "normalize terms"
    return (strip_punct(remove_accents(t)) for t in terms)


def strip_tail(name):
    "get rid of all trailing non-letter symbols except the dot"
    match = re.search(tail_removal_rexp, name)
    if match is not None:
        name = name[: match.span()[0]]
    return name


def normalized(text):
    "caseless Unicode normalization"
    return remove_accents(text)


def prepare_default_terms():
    "construct an optimized term structure for basename extraction"
    terms = get_unique_terms()
    nterms = normalize_terms(terms)
    ntermparts = (t.split() for t in nterms)
    # make sure that the result is deterministic, sort terms descending by number of tokens, ascending by names
    sntermparts = sorted(ntermparts, key=lambda x: (-len(x), x))
    return [(len(tp), tp) for tp in sntermparts]


def custom_basename(name, terms, suffix=True, prefix=False, middle=False, **kwargs):
    "return cleaned base version of the business name"

    name = strip_tail(name)
    nparts = name.split()
    nname = normalized(name)
    nnparts = list(map(strip_punct, nname.split()))
    nnsize = len(nnparts)

    if suffix:
        for termsize, termparts in terms:
            if nnparts[-termsize:] == termparts:
                del nnparts[-termsize:]
                del nparts[-termsize:]

    if prefix:
        for termsize, termparts in terms:
            if nnparts[:termsize] == termparts:
                del nnparts[:termsize]
                del nparts[:termsize]

    if middle:
        for termsize, termparts in terms:
            if termsize > 1:
                sizediff = nnsize - termsize
                if sizediff > 1:
                    for i in range(0, nnsize-termsize+1):
                        if termparts == nnparts[i:i+termsize]:
                            del nnparts[i:i+termsize]
                            del nparts[i:i+termsize]
            else:
                if termparts[0] in nnparts[1:-1]:
                    idx = nnparts[1:-1].index(termparts[0])
                    del nnparts[idx+1]
                    del nparts[idx+1]


    return strip_tail(" ".join(nparts))


# convenience for most common use cases that don't parametrize base name extraction
basename = functools.partial(custom_basename, terms=prepare_default_terms())

In [6]:
terms = prepare_default_terms()

In [7]:
#Ensure Brackets are handled correctly
basename("Net Current Assets (Alpha IV)")

'Net Current Assets (Alpha IV)'

In [8]:
basename('11143646 Canada Inc')

'11143646 Canada'

In [9]:
basename('@hand Software Corporation')

'@hand Software'

In [10]:
basename('24 Mobile Advertsiing Solutions AB')

'24 Mobile Advertsiing Solutions'

## Company Names

In [ ]:
filename = 'OriginalCompanyNames.xlsx'
path = f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{filename}'

In [ ]:
#Company Names
df = pd.read_excel(path)

In [ ]:
df['Cleaned Name'] = ''

In [ ]:
df.head()

In [ ]:
#from cleanco import basename

In [ ]:
#Apply basename function to each company name
for index, row in df.iterrows():
    df['Cleaned Name'][index] = basename(row['Original Company Name'])

In [ ]:
df.head()

In [ ]:
# saved_name = 'CleanedCompanyNames.xlsx'
# path =  f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{saved_name}'
# df.to_excel(path,index=False)

In [11]:
#Read in saved cleaned names
saved_name = 'CleanedCompanyNames.xlsx'
path =  f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{saved_name}'
df = pd.read_excel(path)

In [12]:
df.head()

,Original Company Name,Cleaned Name
0,@hand Software Corporation,@hand Software
1,[24]7.ai,[24]7.ai
2,[24]7.ai,[24]7.ai
3,1 Beekman LLC,1 Beekman
4,1 Beekman LLC,1 Beekman


## Clustering

In [13]:
company_names = df['Cleaned Name'].to_list()

In [14]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(company_names)

In [15]:
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.3)
labels = clustering.fit_predict(x.toarray())

In [16]:
labels

array([6047, 2213, 2213, ...,    7,    7,    7], dtype=int64)

In [17]:
df['Cluster_labels']= labels

In [18]:
df.head()

,Original Company Name,Cleaned Name,Cluster_labels
0,@hand Software Corporation,@hand Software,6047
1,[24]7.ai,[24]7.ai,2213
2,[24]7.ai,[24]7.ai,2213
3,1 Beekman LLC,1 Beekman,2279
4,1 Beekman LLC,1 Beekman,2279


In [20]:
# saved_name = 'ClusteredCompanyNames.xlsx'
# path =  f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{saved_name}'
# df.to_excel(path,index=False)

## Standardising Clusters

In [ ]:
#Load Saved Cluster Data
# saved_name = 'ClusteredCompanyNames.xlsx'
# path =  f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{saved_name}'
# df = pd.read_excel(path)

In [21]:
# Define a function to propose the standardized name for each cluster
def propose_standardised_name(group):
    unique_values = group['Cleaned Name'].unique()
    if len(unique_values) == 1:
        return pd.DataFrame({'Proposed Standardised Name': [unique_values[0]]})
    else:
        return pd.DataFrame({'Proposed Standardised Name': ['']})

# Group the DataFrame by the 'Cluster Label' and propose standardized names for each cluster
proposed_names_df = df.groupby('Cluster_labels').apply(propose_standardised_name).reset_index(level=1, drop=True)
#Merge proposed names
df = df.merge(proposed_names_df, left_on='Cluster_labels', right_index=True)

In [22]:
#Visualise Results
for label in df['Cluster_labels'].unique():
    if len(df[df['Cluster_labels']==label]['Cleaned Name'].unique())>1:
        display(len(df[df['Cluster_labels']==label]['Cleaned Name'].unique()))
        grouping = df[df['Cluster_labels']==label]
        display(grouping)
   

2

,Original Company Name,Cleaned Name,Cluster_labels,Proposed Standardised Name
90,800APP,800APP,2278,
91,800App,800App,2278,


2

,Original Company Name,Cleaned Name,Cluster_labels,Proposed Standardised Name
249,Actis Emerging Markets 3,Actis Emerging Markets 3,10,
7826,Net Curr (Actis Emerging Markets 3),Net Curr (Actis Emerging Markets 3),10,


3

...

,Original Company Name,Cleaned Name,Cluster_labels,Proposed Standardised Name
15447,YeePay,YeePay,15,
15448,YeePay,YeePay,15,
15449,YeePay,YeePay,15,
15450,Yeepay,Yeepay,15,
15451,Yeepay,Yeepay,15,


2

,Original Company Name,Cleaned Name,Cluster_labels,Proposed Standardised Name
15554,Zoominfo,Zoominfo,3759,
15555,ZoomInfo,ZoomInfo,3759,
15556,ZoomInfo,ZoomInfo,3759,
15557,ZoomInfo,ZoomInfo,3759,


2

,Original Company Name,Cleaned Name,Cluster_labels,Proposed Standardised Name
15558,ZoomInfo (Tax Receivable),ZoomInfo (Tax Receivable),5931,
15559,ZoomInfo Tax receivable,ZoomInfo Tax receivable,5931,


In [23]:
saved_name = 'StandardisedCompanyNames.xlsx'
path =  f'C:/Users/aneave/OneDrive/Documents/Naming Conventions/Standardising Company Names/{saved_name}'
df.to_excel(path,index=False)